In [1]:
%matplotlib inline
import torch
from torch import nn
from torch.nn import functional as F
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import utils
import torchvision
from torchvision.io import image
from torchvision.transforms.functional import to_pil_image
import pandas as pd
import time
import numpy as np
import collections
import re
import random
import math

In [2]:
batch_size, num_steps = 32, 35
# train_iter, vocab = utils.load_data_txt(utils.sanguo_txt_path, batch_size, num_steps)
train_iter, vocab = utils.load_data_txt(utils.santi_txt_path, batch_size, num_steps)
print(len(vocab))

1464


In [5]:
# 初始化模型参数
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01

    def init_three_params():
        return (normal((num_inputs, num_hiddens)), normal((num_hiddens, num_hiddens)), torch.zeros(num_hiddens, device=device))

    W_xz, W_hz, b_z = init_three_params()  # 更新门参数
    W_xr, W_hr, b_r = init_three_params()  # 重置门参数
    W_xh, W_hh, b_h = init_three_params()  # 候选隐状态参数
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    # 附加梯度
    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

In [6]:
# 隐状态的初始化
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [7]:
# 定义门控循环单元模型
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid((X @ W_xz) + (H @ W_hz) + b_z)
        R = torch.sigmoid((X @ W_xr) + (H @ W_hr) + b_r)
        H_tilda = torch.tanh((X @ W_xh) + ((R * H) @ W_hh) + b_h)    # 候选状态(使用重置门更新候选状态)
        H = Z * H + (1 - Z) * H_tilda    # 使用更新门、上一时间隐状态、候选状态来更新隐状态
        Y = H @ W_hq + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)

In [9]:
# predict_prefix = ['cao cao', 'kingdom']
predict_prefix = ['三体组织', '物理学']

In [10]:
# 训练和预测
vocab_size, num_hiddens, device = len(vocab), 256, utils.try_gpu()
num_epochs, lr = 500, 1
model = utils.RNNModelScratch(len(vocab), num_hiddens, device, get_params, init_gru_state, gru)
utils.train_ch8(model, train_iter, vocab, lr, num_epochs, device, predict_prefix=predict_prefix)

三体组织的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
epoch: 50/500, ppl: 483.85122701489234
三体组织，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这，这
epoch: 100/500, ppl: 463.60497411308666
三体组织的，这是的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个
epoch: 150/500, ppl: 391.4283088902171
三体组织，但是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一个个，我们是一
epoch: 200/500, ppl: 276.3856256154485
三体组织的物理学学学————一个物学学学的物质

# 简洁实现

In [11]:
num_inputs = vocab_size
gru_layer = nn.GRU(num_inputs, num_hiddens)
model = utils.RNNModel(gru_layer, len(vocab))
model = model.to(device)
utils.train_ch8(model, train_iter, vocab, lr, num_epochs, device, predict_prefix=predict_prefix)

三体组织的，的，的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的，的，的的，的的
epoch: 50/500, ppl: 463.7524473040788
三体组织的，但是的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个的，这个
epoch: 100/500, ppl: 357.43361128196756
三体组织，但是一个个大的，他们是一个个大，他们是一个人。”“我是一个个大，他们是一个人。”“我是一个个大，他们是一个人。”“我是一个个大，他们是一个人。”“我是一个个大，他们是一个人。”“我是一个个大，他们是一个人。”“我是一个个大，他们是一个人。”“我是一个个大，他们是一个人。”“我是一个个大，他们是一个人。”“我是一个个大，他们是一个人。”“我是一个个大，他们是一个人。”“我是一个个大，他们是一个人。
epoch: 150/500, ppl: 213.86113048409538
三体组织，这个个个大的工率。”“我是这个人度的是一个发现的，他们的研究中，就是一个人度的，他们是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度的，他是一个人度
epoch: 200/500, ppl: 94.55736938172524
三体组织，他们这个人类还在很大，但至这样计算机

# 门控循环神经网络可以更好地捕获时间步距离很长的序列上的依赖关系。

# 重置门有助于捕获序列中的短期依赖关系。

# 更新门有助于捕获序列中的长期依赖关系。

# 重置门打开时，门控循环单元包含基本循环神经网络；更新门打开时，门控循环单元可以跳过子序列。